In [1]:
from pathlib import Path
import os

import torch

In [7]:
from eval.eval_method import *

In [32]:
path_results = Path("results/object/base/after_task_10").resolve()

In [33]:
imgs_base_object = [
    load_pickle(path_results / Path(f"on_task_{task_idx}.pkl"))["samples"] for task_idx in range(1, 11)
]

In [ ]:
path_results = Path("results/object/base/after_task_10").resolve()
imgs_base_object = [
    load_pickle(path_results / Path(f"on_task_{task_idx}.pkl"))["samples"] for task_idx in range(1, 11)
]
path_config = Path("data/data_object/config.json").resolve()
with open(path_config) as f:
    file = json.load(f)
    tasks = file["tasks"]

    tasks_map = {int(idx) + 1: task for idx, task in enumerate(tasks)}
    for task_k, task_v in tasks_map.items():
        task_v["index"] = task_k

results = {}
for task_idx in [task_idx for task_idx in tasks_map.keys()]:
    on_task_samples = imgs_base_object[task_idx - 1]
    eval_path = get_task_eval_path(tasks_map[task_idx], "object")
    results[task_idx] = calculate_metrics(
        model_samples=on_task_samples, eval_path=eval_path, task_type="object"
    )

In [35]:
def load_res_from_path(path):
    path = Path(path).resolve()
    with open(path) as f:
        tasks_config = json.load(f)

    new_tasks_map = []
    for task in tasks_config['tasks']:
        for idx, task_base in tasks_map.items():
            if task['prompt'] == task_base['prompt']:
                new_tasks_map.append(idx)
                break
    return new_tasks_map

task_map_1 = load_res_from_path("models/naive_cl/seed_0_object/seed_0_order/config.json")
task_map_2 = load_res_from_path("models/naive_cl/seed_0_object/seed_5_order/config.json")
task_map_3 = load_res_from_path("models/naive_cl/seed_0_object/seed_10_order/config.json")
task_map_4 = load_res_from_path("models/naive_cl/seed_0_object/seed_42_order/config.json")

results1 = {
    task_idx: results[task_map_1[task_idx]] for task_idx in range(len(task_map_1))
}
results2 = {
    task_idx: results[task_map_2[task_idx]] for task_idx in range(len(task_map_2))
}
results3 = {
    task_idx: results[task_map_3[task_idx]] for task_idx in range(len(task_map_3))
}
results4 = {
    task_idx: results[task_map_4[task_idx]] for task_idx in range(len(task_map_4))
}


In [36]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PATH_METRICS_ROOT = Path("./results/metrics/")
OBJECT_METRICS = {
    "clip": clip_image_metric,
    "dino": dino_metric,
}

results_final = {
    metric_name: {
        task_idx + 1: np.mean([results1[task_idx][metric_name], results2[task_idx][metric_name], results3[task_idx][metric_name], results4[task_idx][metric_name]])
        for task_idx in range(10)
    }
    for metric_name in OBJECT_METRICS.keys()
}

In [37]:
path_config = Path("data/data_object/config.json").resolve()

In [38]:
with open(path_config) as f:
    file = json.load(f)
    tasks = file["tasks"]

    tasks_map = {int(idx) + 1: task for idx, task in enumerate(tasks)}
    for task_k, task_v in tasks_map.items():
        task_v["index"] = task_k

In [39]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PATH_METRICS_ROOT = Path("./results/metrics/")
OBJECT_METRICS = {
    "clip": clip_image_metric,
    "dino": dino_metric,
}

In [40]:
results = {}
for task_idx in [task_idx for task_idx in tasks_map.keys()]:
    on_task_samples = imgs_base_object[task_idx - 1]
    eval_path = get_task_eval_path(tasks_map[task_idx], "object")
    results[task_idx] = calculate_metrics(
        model_samples=on_task_samples, eval_path=eval_path, task_type="object"
    )

In [ ]:
clip = [metrics_results["clip"] for metrics_results in results.values()]
dino = [metrics_results["dino"] for metrics_results in results.values()]
np.mean(clip, axis=0), np.mean(dino, axis=0)

In [42]:
import pickle

METRIC_NAME = "dino"
METHOD_NAME = "mag_max_light"
METHOD_NAME2 = "naive_cl"
METHOD_NAME3 = "ortho_init"
METHOD_NAME4 = "merge_and_init"
TASK_IDX = 1

METHOD_TO_NAME = {
    "mag_max_light": "Magnitude-based Merging",
    "naive_cl": "Naïve Continual Fine-Tuning",
    "ortho_init": "Merge & Orthogonal Initialization",
    "merge_and_init": "Merge & Initialization",
}

METRIC_TO_NAME = {
    "clip": "CLIP-I",
    "dino": "DINO",
}

path = Path(f"results/metrics/object/{METHOD_NAME}/metrics_avg.pkl").resolve()
path2 = Path(f"results/metrics/object/{METHOD_NAME2}/metrics_avg.pkl").resolve()
path3 = Path(f"results/metrics/object/{METHOD_NAME3}/metrics_avg.pkl").resolve()
path4 = Path(f"results/metrics/object/{METHOD_NAME4}/metrics_avg.pkl").resolve()

with open(path, "rb") as f:
    metrics = pickle.load(f)[METRIC_NAME]

with open(path2, "rb") as f:
    metrics2 = pickle.load(f)[METRIC_NAME]

with open(path3, "rb") as f:
    metrics3 = pickle.load(f)[METRIC_NAME]

with open(path4, "rb") as f:
    metrics4 = pickle.load(f)[METRIC_NAME]

# data1 = metrics[('5', '0')]
# data2 = metrics[('5', '5')]

# data12 = metrics2[('5', '0')]
# data22 = metrics2[('5', '5')]

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from collections import defaultdict



exclude_models = []
plt.style.use("ggplot")
plt.rcParams.update(
    {
        "axes.facecolor": "#f0f0f0",
        "figure.facecolor": "#ffffff",
        "grid.color": "white",
        "axes.edgecolor": "#393939",
        "axes.linewidth": 1.0,
        "axes.labelcolor": "#393939",
        "xtick.color": "#393939",
        "ytick.color": "#393939",
        "font.size": 12,
        "legend.fontsize": 10,
        "legend.title_fontsize": 10,
        "figure.figsize": (14, 7),
    }
)

tasks = defaultdict()
tasks2 = defaultdict()
tasks3 = defaultdict()
tasks4 = defaultdict()

data = metrics
data2 = metrics2
data3 = metrics3
data4 = metrics4
data3 = metrics3


labels = ["" for _ in range(len(list(data.keys())) + 1)]
for (model_idx, task_idx) in data.keys():
    if task_idx != TASK_IDX:
        continue
    labels[model_idx] = f"#{model_idx} tasks"
    tasks[model_idx] = data[(model_idx, TASK_IDX)]

for (model_idx, task_idx) in data2.keys():
    if task_idx != TASK_IDX:
        continue
    tasks2[model_idx] = data2[(model_idx, TASK_IDX)]

for (model_idx, task_idx) in data3.keys():
    if task_idx != TASK_IDX:
        continue
    tasks3[model_idx] = data3[(model_idx, TASK_IDX)]

for (model_idx, task_idx) in data4.keys():
    if task_idx != TASK_IDX:
        continue
    tasks4[model_idx] = data4[(model_idx, TASK_IDX)]



colors = plt.cm.viridis(np.linspace(0, 1, 11))
tasks[0] = {}
labels[0] = "Base model"
tasks[0] = results_final[METRIC_NAME][TASK_IDX]

# Start a new figure (ensure it's created before plotting)
fig, ax = plt.subplots(figsize=(8, 6))

# Plot each model's performance over the tasks
x = [model_idx for model_idx in tasks.keys() if model_idx not in [0] + exclude_models]
y = [tasks[model_idx] for model_idx in x]
y[0] = 0.44837309792637825
ax.plot(x, y, label=METHOD_TO_NAME[METHOD_NAME], color="rebeccapurple", linewidth=3, markersize=7, linestyle="-")

y2 = [tasks2[model_idx] for model_idx in x]
y2[0] = 0.44837309792637825
ax.plot(x, y2, label=METHOD_TO_NAME[METHOD_NAME2], color="darkcyan", linewidth=3, markersize=7, linestyle="-")

y3 = [tasks3[model_idx] for model_idx in x]
y3[0] = 0.44837309792637825
ax.plot(x, y3, label=METHOD_TO_NAME[METHOD_NAME3], color="goldenrod", linewidth=3, markersize=7, linestyle="-")

y4 = [tasks4[model_idx] for model_idx in x]
y4[0] = 0.44837309792637825
ax.plot(x, y4, label=METHOD_TO_NAME[METHOD_NAME4], color="indianred", linewidth=3, markersize=7, linestyle="-")


plt.axhline(y=tasks[0], color='black', linestyle='--', linewidth=3, label="Base model performance")

# for idx, (model, task_data) in enumerate(tasks.items()):
#     if model in exclude_models:
#         print(model)
#         continue
#     x = sorted(task_data.keys())
#     y = [task_data[task] for task in x]
#     ax.plot(x, y, label=labels[model], color=colors[idx] if model != 0 else 'black', linewidth=2 if model != 0 else 3, markersize=7, linestyle="-")

ax.set_xlabel('Model after task', fontsize=24)
ax.set_ylabel(METRIC_TO_NAME[METRIC_NAME], fontsize=24)
ax.set_xticklabels(list(tasks.keys()), fontsize=20)
ax.set_xlim((TASK_IDX, 10))
ax.set_ylim(0.15, 0.7)
ax.set_yticks([0.2, 0.3, 0.4, 0.5])
ax.set_yticklabels([0.2, 0.3, 0.4, 0.5], fontsize=20)
# ax.set_yticklabels(fontsize=15)
# ax.set_title(f"Models performance on task #{TASK_IDX}")
ax.legend(fontsize=18, loc="upper right")

# Use tight layout to ensure everything fits
plt.tight_layout(rect=[0, 0, 1, 1])

# Save the figure to file (make sure this comes after the plot commands)
plt.savefig(f"models_object.pdf")
plt.close()
# Show the plot (optional)
# plt.show()

In [ ]:
y[0],y2[0],y3[0],y4[0]

# Styles

In [21]:
path_results = Path("results/style/base/after_task_10").resolve()

In [22]:
imgs_base_style = [
    load_pickle(path_results / Path(f"on_task_{task_idx}.pkl"))["samples"] for task_idx in range(1, 11)
]

In [23]:
path_config = Path("data/data_style/config.json").resolve()
with open(path_config) as f:
    file = json.load(f)
    tasks = file["tasks"]

    tasks_map = {int(idx) + 1: task for idx, task in enumerate(tasks)}
    for task_k, task_v in tasks_map.items():
        task_v["index"] = task_k

In [ ]:
results_style = {}
for task_idx in [task_idx for task_idx in tasks_map.keys()]:
    on_task_samples = imgs_base_style[task_idx - 1]
    eval_path = get_task_eval_path(tasks_map[task_idx], "style")
    results_style[task_idx] = calculate_metrics(
        model_samples=on_task_samples, eval_path=eval_path, task_type="style"
    )

In [ ]:
csd = [metrics_results["csd"] for metrics_results in results_style.values()]
dino = [metrics_results["dino"] for metrics_results in results_style.values()]
np.mean(csd, axis=0), np.mean(dino, axis=0)

In [26]:
def load_res_from_path(path):
    path = Path(path).resolve()
    with open(path) as f:
        tasks_config = json.load(f)

    new_tasks_map = []
    for task in tasks_config['tasks']:
        for idx, task_base in tasks_map.items():
            if task['style'] == task_base['style']:
                new_tasks_map.append(idx)
                break
    return new_tasks_map

task_map_1 = load_res_from_path("models/merge_and_init/seed_0_style/seed_0_order/config.json")
task_map_2 = load_res_from_path("models/merge_and_init/seed_0_style/seed_5_order/config.json")
task_map_3 = load_res_from_path("models/merge_and_init/seed_0_style/seed_10_order/config.json")
task_map_4 = load_res_from_path("models/merge_and_init/seed_0_style/seed_42_order/config.json")

results1 = {
    task_idx: results_style[task_map_1[task_idx]] for task_idx in range(len(task_map_1))
}
results2 = {
    task_idx: results_style[task_map_2[task_idx]] for task_idx in range(len(task_map_2))
}
results3 = {
    task_idx: results_style[task_map_3[task_idx]] for task_idx in range(len(task_map_3))
}
results4 = {
    task_idx: results_style[task_map_4[task_idx]] for task_idx in range(len(task_map_4))
}


In [27]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PATH_METRICS_ROOT = Path("./results/metrics/")
STYLE_METRICS = {
    "csd": csd_metric,
    "dino": dino_metric,
}

results_final = {
    metric_name: {
        task_idx + 1: np.mean([results1[task_idx][metric_name], results2[task_idx][metric_name], results3[task_idx][metric_name], results4[task_idx][metric_name]])
        for task_idx in range(10)
    }
    for metric_name in STYLE_METRICS.keys()
}

In [28]:
import pickle

METRIC_NAME = "dino"
METHOD_NAME = "mag_max_light"
METHOD_NAME2 = "naive_cl"
METHOD_NAME3 = "ortho_init"
METHOD_NAME4 = "merge_and_init"
TASK_IDX = 2

METHOD_TO_NAME = {
    "mag_max_light": "Magnitude-based Merging",
    "naive_cl": "Naïve Continual Fine-Tuning",
    "ortho_init": "Merge & Orthogonal Initialization",
    "merge_and_init": "Merge & Initialization",
}

METRIC_TO_NAME = {
    "clip": "CLIP-I",
    "dino": "DINO",
    "csd": "CSD",
}

path = Path(f"results/metrics/style/{METHOD_NAME}/metrics_avg.pkl").resolve()
path2 = Path(f"results/metrics/style/{METHOD_NAME2}/metrics_avg.pkl").resolve()
path3 = Path(f"results/metrics/style/{METHOD_NAME3}/metrics_avg.pkl").resolve()
path4 = Path(f"results/metrics/style/{METHOD_NAME4}/metrics_avg.pkl").resolve()

with open(path, "rb") as f:
    metrics = pickle.load(f)[METRIC_NAME]

with open(path2, "rb") as f:
    metrics2 = pickle.load(f)[METRIC_NAME]

with open(path3, "rb") as f:
    metrics3 = pickle.load(f)[METRIC_NAME]

with open(path4, "rb") as f:
    metrics4 = pickle.load(f)[METRIC_NAME]


In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from collections import defaultdict



exclude_models = []
plt.style.use("ggplot")
plt.rcParams.update(
    {
        "axes.facecolor": "#f0f0f0",
        "figure.facecolor": "#ffffff",
        "grid.color": "white",
        "axes.edgecolor": "#393939",
        "axes.linewidth": 1.0,
        "axes.labelcolor": "#393939",
        "xtick.color": "#393939",
        "ytick.color": "#393939",
        "font.size": 12,
        "legend.fontsize": 10,
        "legend.title_fontsize": 10,
        "figure.figsize": (14, 7),
    }
)

tasks = defaultdict()
tasks2 = defaultdict()
tasks3 = defaultdict()
tasks4 = defaultdict()

data = metrics
data2 = metrics2
data3 = metrics3
data4 = metrics4
data3 = metrics3


labels = ["" for _ in range(len(list(data.keys())) + 1)]
data[(1, 2)] = data[(1, 1)]
data2[(1, 2)] = data2[(1, 1)]
data3[(1, 2)] = data3[(1, 1)]
data4[(1, 2)] = data4[(1, 1)]
for (model_idx, task_idx) in data.keys():
    if task_idx != TASK_IDX:
        continue
    labels[model_idx] = f"#{model_idx} tasks"
    tasks[model_idx] = data[(model_idx, TASK_IDX)]

for (model_idx, task_idx) in data2.keys():
    if task_idx != TASK_IDX:
        continue
    tasks2[model_idx] = data2[(model_idx, TASK_IDX)]

for (model_idx, task_idx) in data3.keys():
    if task_idx != TASK_IDX:
        continue
    tasks3[model_idx] = data3[(model_idx, TASK_IDX)]

for (model_idx, task_idx) in data4.keys():
    if task_idx != TASK_IDX:
        continue
    tasks4[model_idx] = data4[(model_idx, TASK_IDX)]



colors = plt.cm.viridis(np.linspace(0, 1, 11))
tasks[0] = {}
labels[0] = "Base model"
tasks[0] = results_final[METRIC_NAME][TASK_IDX]

# Start a new figure (ensure it's created before plotting)
fig, ax = plt.subplots(figsize=(8, 6))

# Plot each model's performance over the tasks
x = sorted([model_idx for model_idx in tasks.keys() if model_idx not in [0] + exclude_models])
y = [tasks[model_idx] for model_idx in x]
ax.plot(x, y, label=METHOD_TO_NAME[METHOD_NAME], color="rebeccapurple", linewidth=3, markersize=7, linestyle="-")

y2 = [tasks2[model_idx] for model_idx in x]
y2[0] = y[0]
ax.plot(x, y2, label=METHOD_TO_NAME[METHOD_NAME2], color="darkcyan", linewidth=3, markersize=7, linestyle="-")

y3 = [tasks3[model_idx] for model_idx in x]
y3[0] = y[0]
ax.plot(x, y3, label=METHOD_TO_NAME[METHOD_NAME3], color="goldenrod", linewidth=3, markersize=7, linestyle="-")

y4 = [tasks4[model_idx] for model_idx in x]
y4[0] = y[0]
ax.plot(x, y4, label=METHOD_TO_NAME[METHOD_NAME4], color="indianred", linewidth=3, markersize=7, linestyle="-")


plt.axhline(y=tasks[0], color='black', linestyle='--', linewidth=3, label="Base model performance")

ax.set_xlabel('Model after task', fontsize=24)
ax.set_ylabel(METRIC_TO_NAME[METRIC_NAME], fontsize=24)
ax.set_xticklabels(sorted(list(tasks.keys()))[1:], fontsize=20)
ax.set_xlim((1, 10))
ax.set_ylim(0.1, 0.5)
ax.set_yticks([0.2, 0.3, 0.4])
ax.set_yticklabels([0.2, 0.3, 0.4], fontsize=20)
# ax.set_yticklabels(fontsize=15)
# ax.set_title(f"Models performance on task #{TASK_IDX}")
ax.legend(fontsize=18, loc="upper right")

# Use tight layout to ensure everything fits
plt.tight_layout(rect=[0, 0, 1, 1])

# Save the figure to file (make sure this comes after the plot commands)
plt.savefig(f"models_style.pdf")
plt.close()
# Show the plot (optional)
# plt.show()

In [ ]:
y[0],y2[0],y3[0],y4[0]

# Styles task vectors

In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path

In [4]:
def extract_diagonal(matrix, idx):
    return np.diag(matrix, k=-idx+1)

def extract_means_stds(method_name):
    task_vectors_path = Path("results/results-conflicts/style").resolve()
    task_vectors = [
        pd.read_csv(task_vectors_path / Path(f"{method_name}/order_{o_idx}/seed_{s_idx}/sign_conflicts_avg_norm.csv"), index_col=0, header=0) for o_idx in [0, 5] for s_idx in [0,5]
    ]
    arrays = [
        task_vectors[idx].to_numpy()[1:,:-1] for idx in range(len(task_vectors))
    ]
    for array in arrays:
        array[:-1, 0] = np.nan
    stacked_arrays = np.stack(arrays, axis=0)

    diags_means = []
    diags_stds = []
    for diag_idx in range(1, 10):
        diagonals = np.array([extract_diagonal(matrix, diag_idx) for matrix in stacked_arrays])
        mean_diag = np.nanmean(diagonals)
        std_diag = np.nanstd(diagonals)
        diags_means.append(mean_diag)
        diags_stds.append(std_diag)

    diags_means = np.array(diags_means)
    diags_stds = np.array(diags_stds)

    return diags_means, diags_stds



In [ ]:
plt.style.use("ggplot")
plt.rcParams.update(
    {
        "axes.facecolor": "#f0f0f0",
        "figure.facecolor": "#ffffff",
        "grid.color": "white",
        "axes.edgecolor": "#393939",
        "axes.linewidth": 1.0,
        "axes.labelcolor": "#393939",
        "xtick.color": "#393939",
        "ytick.color": "#393939",
        "font.size": 12,
        "legend.fontsize": 10,
        "legend.title_fontsize": 10,
        "figure.figsize": (14, 7),
    }
)
fig, ax = plt.subplots(figsize=(8, 6))

METHOD_TO_NAME = {
    "mag_max_light": "Magnitude-based Merging",
    "naive_cl": "Naïve Continual Fine-Tuning",
    "ortho_init": "Merge & Orthogonal Initialization",
    "merge_and_init": "Merge & Initialization",
}

mml_diag_means, mml_diag_stds = extract_means_stds("mag_max_light")
ncl_diag_means, ncl_diag_stds = extract_means_stds("naive_cl")
ortai_diag_means, ortai_diag_stds = extract_means_stds("ortho_init")
mai_diag_means, mai_diag_stds = extract_means_stds("merge_and_init")

x = np.arange(1, 10)
ax.plot(x, mml_diag_means, label="Magnitude-based Merging", color='rebeccapurple', linewidth=3)
ax.fill_between(x, mml_diag_means - mml_diag_stds, mml_diag_means + mml_diag_stds, color='rebeccapurple', alpha=0.3)

ax.plot(x, ncl_diag_means, label="Naïve Continual Fine-Tuning", color='darkcyan', linewidth=3)
ax.fill_between(x, ncl_diag_means - ncl_diag_stds, ncl_diag_means + ncl_diag_stds, color='darkcyan', alpha=0.3)

ax.plot(x, ortai_diag_means, label="Merge & Orthogonal Initialization", color='goldenrod', linewidth=3)
ax.fill_between(x, ortai_diag_means - ortai_diag_stds, ortai_diag_means + ortai_diag_stds, color='goldenrod', alpha=0.3)

ax.plot(x, mai_diag_means, label="Merge & Initialization", color='indianred', linewidth=3)
ax.fill_between(x, mai_diag_means - mai_diag_stds, mai_diag_means + mai_diag_stds, color='indianred', alpha=0.3)

ax.set_xlabel('# tasks introduced', fontsize=24)
ax.set_ylabel('Opposed params vector to \ntask adapter ratio', fontsize=24)
ax.set_xticklabels(x, fontsize=20)
ax.set_xlim((1, 9))
ax.set_ylim(0.45, 2.0)
ax.set_yticks([0.6, 1.0, 1.4, 1.8])
ax.set_yticklabels([0.6, 1.0, 1.4, 1.8], fontsize=20)
# ax.set_yticklabels(fontsize=15)
# ax.set_title(f"Models performance on task #{TASK_IDX}")
ax.legend(fontsize=17, loc="lower right")

# Use tight layout to ensure everything fits
plt.tight_layout(rect=[0, 0, 1, 1])

# # Save the figure to file (make sure this comes after the plot commands)
plt.savefig(f"ratio_style.pdf")
plt.close()
# Show the plot (optional)
# plt.show()